# Building a Neural Network from Scratch

Our NN will have a simple two-layer architecture. Input layer $a^{[0]}$ will have 784 units corresponding to the 784 pixels in each 28x28 input image. A hidden layer $a^{[1]}$ will have 10 units with ReLU activation, and finally our output layer $a^{[2]}$ will have 10 units corresponding to the ten digit classes with softmax activation.

**Forward propagation**

$$Z^{[1]} = W^{[1]} X + b^{[1]}$$
$$A^{[1]} = g_{\text{ReLU}}(Z^{[1]}))$$
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
$$A^{[2]} = g_{\text{softmax}}(Z^{[2]})$$

**Backward propagation**

$$dZ^{[2]} = A^{[2]} - Y$$
$$dW^{[2]} = \frac{1}{m} dZ^{[2]} A^{[1]T}$$
$$dB^{[2]} = \frac{1}{m} \Sigma {dZ^{[2]}}$$
$$dZ^{[1]} = W^{[2]T} dZ^{[2]} .* g^{[1]\prime} (z^{[1]})$$
$$dW^{[1]} = \frac{1}{m} dZ^{[1]} A^{[0]T}$$
$$dB^{[1]} = \frac{1}{m} \Sigma {dZ^{[1]}}$$

**Parameter updates**

$$W^{[2]} := W^{[2]} - \alpha dW^{[2]}$$
$$b^{[2]} := b^{[2]} - \alpha db^{[2]}$$
$$W^{[1]} := W^{[1]} - \alpha dW^{[1]}$$
$$b^{[1]} := b^{[1]} - \alpha db^{[1]}$$

**Vars and shapes**

Forward prop

- $A^{[0]} = X$: 784 x m
- $Z^{[1]} \sim A^{[1]}$: 10 x m
- $W^{[1]}$: 10 x 784 (as $W^{[1]} A^{[0]} \sim Z^{[1]}$)
- $B^{[1]}$: 10 x 1
- $Z^{[2]} \sim A^{[2]}$: 10 x m
- $W^{[1]}$: 10 x 10 (as $W^{[2]} A^{[1]} \sim Z^{[2]}$)
- $B^{[2]}$: 10 x 1

Backprop

- $dZ^{[2]}$: 10 x m ($~A^{[2]}$)
- $dW^{[2]}$: 10 x 10
- $dB^{[2]}$: 10 x 1
- $dZ^{[1]}$: 10 x m ($~A^{[1]}$)
- $dW^{[1]}$: 10 x 10
- $dB^{[1]}$: 10 x 1


# 0. Importing packages


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Using a relative path instead of an absolute
path = r'./digit-recognizer/train.csv'

data = pd.read_csv(path)


Loaded the data into a pandas dataframe.


In [3]:
data.head()


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Converting the data from a dataframe to a numpy array.


In [4]:
data = np.array(data)


Making sure that the model isn't overfitted, i.e., the model makes fairly accurate predictions for the training data but isn't generalised for the data it's supposed to have a high accuracy for. Setting aside a portion of the training data to perform cross-validation on to avoid overfitting.


Shuffling the data before we split the data into dev and training data. Note, `np.random.shuffle()` permutes the sequence in place.


In [5]:
np.random.shuffle(data)


In [6]:
data


array([[6, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [8, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [7]:
# Storing the dimensions
m, n = data.shape

# m - Number of images; n - label + pixels for each image
m, n


(42000, 785)

Splitting the data into dev and training. We're using dev to cross validate and we're setting aside only 1000 images to do so.


In [8]:
# Transposing the data using only 1000 images
data_dev = data[:1000].T

# Storing the labels in YDev
Y_dev = data_dev[0]

# Storing the pixels
X_dev = data_dev[1:]
X_dev = X_dev / 255


In [9]:
# Storing the rest of the images
data_train = data[1000:m].T

# Extract labels
Y_train = data_train[0]

# Get the rest
X_train = data_train[1:]
X_train = X_train / 255


Printing details of all arrays implemented so far.


In [29]:
print(
    f'Printing dimensions of all existing arrays:\n(i) X - pixels\nX_dev: {X_dev.shape}\nX_train: {X_train.shape}\n{X_dev}\n\n(ii) Y - labels\nY_dev: {Y_dev.shape}\nY_train: {Y_train.shape}\n{Y_dev}')


Printing dimensions of all existing arrays:
(i) X - pixels
X_dev: (784, 1000)
X_train: (784, 41000)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

(ii) Y - labels
Y_dev: (1000,)
Y_train: (41000,)
[6 9 4 9 6 8 6 0 3 0 5 0 2 8 3 4 5 8 5 3 2 6 3 8 5 8 7 8 8 9 9 3 1 2 4 1 2
 7 0 1 4 7 7 6 4 3 7 5 2 2 1 5 3 6 5 8 6 2 7 8 7 8 8 6 8 4 3 5 0 8 1 8 1 5
 2 4 4 9 3 6 5 0 1 0 0 9 3 8 0 7 5 9 1 4 7 2 8 5 1 3 8 1 0 3 2 5 5 3 3 2 7
 9 3 9 7 9 7 7 9 1 0 5 8 5 1 5 5 3 3 7 4 6 5 4 7 5 0 6 8 5 3 0 9 4 7 0 9 4
 8 7 5 3 0 5 8 9 1 9 3 7 1 0 1 8 6 1 8 6 9 3 5 9 8 2 1 7 0 4 4 0 9 2 6 1 4
 5 9 3 6 6 3 4 4 9 1 1 8 1 8 7 2 1 4 6 8 0 9 5 2 7 0 9 0 9 4 6 9 3 5 2 8 2
 4 2 8 2 0 1 9 6 3 6 6 1 5 4 6 0 0 2 6 2 8 2 6 9 1 4 6 8 1 1 4 3 8 6 8 4 6
 4 3 8 5 1 6 4 2 3 5 1 2 0 9 5 6 0 9 1 3 1 9 5 8 8 9 8 4 6 4 2 2 7 6 1 5 5
 8 3 6 5 9 0 3 9 6 0 0 6 4 9 2 0 8 5 5 2 0 2 7 1 4 7 8 1 1 2 9 8 9 8 9 2 2
 4 3 5 8 0 9 5 5 7 

# 1. Defining Initial Parameters


Defining a function to initialise the neural network by creating random weights. We use `rand()` to obtain a random value between 0 and 1 and then we subtract from those values to make sure the range in which our random values lie is `[-0.5, 0.5]`.


In [11]:
def init_params():
    # There's 10 connections for each of the 784 nodes
    W1 = np.random.rand(10, n - 1) - 0.5

    # There's 10 biases
    b1 = np.random.rand(10, 1) - 0.5

    # Similarly,
    # There's 10 connections to 10 output nodes
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5

    return W1, b1, W2, b2


Function implementing the ReLU (rectified linear unit) activation function.


In [12]:
def ReLU(Z):
    # Taking the maximum element-wise using numpy
    return np.maximum(0, Z)


Function implementing the softmax activation function


In [13]:
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    
    # Returning the probability
    return A


# 2. Forward Propagation


Defining a function to implement forward propagation through the neural net.


In [14]:
def forward_propagation(W1, b1, W2, b2, X):
    # Deactivated first layer
    Z1 = W1.dot(X) + b1
    
    # Activating Z1
    A1 = ReLU(Z1)
    
    # Creating the next layer's deactivated input
    Z2 = W2.dot(A1) + b2
    
    # Since the next layer is the output layer, we apply softmax
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2


Function to implement one-hot encoding of Y. This is to represent the target classes as an array instead of a label.


In [15]:
def one_hot_encode(Y):
    # Encoding
    one_hot_encoded_df = pd.get_dummies(Y)

    # Taking the transpose so the columns represent images
    one_hot_encoded_array = np.array(one_hot_encoded_df).T

    return one_hot_encoded_array


Test to illustrate the working of `one_hot_encode(Y)`.


In [16]:
test = Y_train[:20]
test


array([0, 3, 7, 0, 8, 8, 1, 0, 4, 8, 6, 6, 6, 0, 2, 0, 0, 0, 2, 4],
      dtype=int64)

In [17]:
df = pd.get_dummies(test).T
df

ls = np.array(df)
ls


array([[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=uint8)

Comparing it to our function.


In [18]:
one_hot_encode(test)


array([[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=uint8)

Alternative one-hot encoding function.


In [19]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    
    one_hot_Y[np.arange(Y.size), Y] = 1
    
    one_hot_Y = one_hot_Y.T
    
    return one_hot_Y


Function to implement the derivative of ReLU.


In [20]:
def derivative_ReLU(Z):
    # Returning 1 if the value is greater than 0. This is because the slope of the `linear` thing is 1.
    return Z > 0


# 3. Back Propagation


Function to back propagate through the neural network to calculate the differences in the weights and biases.


In [21]:
def back_propagation(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    # one_hot_encoded_Y = one_hot_encode(Y)

    # dZ2 = A2 - one_hot_encoded_Y

    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y

    dW2 = (1 / m) * dZ2.dot(A1.T)
    db2 = (1 / m) * np.sum(dZ2)

    dZ1 = W2.T.dot(dZ2) * derivative_ReLU(Z1)
    dW1 = (1 / m) * dZ1.dot(X.T)
    db1 = (1 / m) * np.sum(dZ1)

    return dW1, db1, dW2, db2


# 4. Updating Parameters


Function to update the parameters using learning rate `alpha`.


In [22]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    
    return W1, b1, W2, b2


# 5. Defining Gradient Descent


In [23]:
def get_predictions(A):
    # Returns the indices of the max values
    return np.argmax(A, 0)


Testing the `get_predictions()` function.


In [24]:
test, get_predictions(test)

(array([0, 3, 7, 0, 8, 8, 1, 0, 4, 8, 6, 6, 6, 0, 2, 0, 0, 0, 2, 4],
       dtype=int64),
 4)

In [25]:
def get_accuracy(predictions, Y):
    print(predictions, Y)
    
    return np.sum(predictions == Y) / Y.size

Gradient descent is an optimization algorithm which is commonly-used to train machine learning models and neural networks. Training data helps these models learn over time, and the cost function within gradient descent specifically acts as a barometer, gauging its accuracy with each iteration of parameter updates. Until the function is close to or equal to zero, the model will continue to adjust its parameters to yield the smallest possible error. Once machine learning models are optimized for accuracy, they can be powerful tools for artificial intelligence (AI) and computer science applications.


In [26]:
def gradient_descent(X, Y, epochs, alpha):
    # Defining weights and biases
    W1, b1, W2, b2 = init_params()

    for i in range(epochs):
        # Step 1
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)

        # Step 2
        dW1, db1, dW2, db2 = back_propagation(Z1, A1, Z2, A2, W2, X, Y)

        # Step 3
        W1, b1, W2, b2 = update_params(
            W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        # Every 10th iteration
        if i % 10 == 0:
            print("Iteration: ", i)

            # A2 is the output from the forward propagation
            predictions = get_predictions(A2)

            print(get_accuracy(predictions, Y))

    return W1, b1, W2, b2


# 6. Running the neural network


Running it on `X_train` and `Y_train`. We run it for 500 epochs with a learning rate of 1. As we can clearly tell, the output displays for each 10 iterations, a rapidly increasing accuracy settling at a nice 91.7%.


In [27]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 1)

Iteration:  0
[2 4 2 ... 2 1 1] [0 3 7 ... 5 8 1]
0.06275609756097561
Iteration:  10
[2 2 7 ... 3 1 1] [0 3 7 ... 5 8 1]
0.4263658536585366
Iteration:  20
[3 8 7 ... 3 3 3] [0 3 7 ... 5 8 1]
0.4087073170731707
Iteration:  30
[6 8 7 ... 5 1 1] [0 3 7 ... 5 8 1]
0.5701951219512195
Iteration:  40
[0 5 7 ... 5 1 3] [0 3 7 ... 5 8 1]
0.7197317073170731
Iteration:  50
[0 3 7 ... 3 1 1] [0 3 7 ... 5 8 1]
0.7190487804878049
Iteration:  60
[0 5 7 ... 5 8 1] [0 3 7 ... 5 8 1]
0.7711219512195122
Iteration:  70
[0 5 7 ... 5 8 1] [0 3 7 ... 5 8 1]
0.7936829268292683
Iteration:  80
[0 0 7 ... 5 1 1] [0 3 7 ... 5 8 1]
0.7110975609756097
Iteration:  90
[0 3 7 ... 5 8 1] [0 3 7 ... 5 8 1]
0.8274634146341463
Iteration:  100
[0 3 7 ... 3 8 1] [0 3 7 ... 5 8 1]
0.8293414634146341
Iteration:  110
[0 3 7 ... 5 8 1] [0 3 7 ... 5 8 1]
0.8640731707317073
Iteration:  120
[0 3 7 ... 3 1 1] [0 3 7 ... 5 8 1]
0.8470243902439024
Iteration:  130
[0 3 7 ... 5 8 1] [0 3 7 ... 5 8 1]
0.8770975609756098
Iteration:  140


Running it on `X_dev` and `Y_dev` to cross-validate the model to overcome overfitting.


In [28]:
W1, b1, W2, b2 = gradient_descent(X_dev, Y_dev, 500, 1)

Iteration:  0
[5 3 3 3 5 6 5 3 3 3 3 3 6 3 3 3 3 3 3 6 8 3 3 6 6 3 3 6 3 3 3 3 6 3 3 7 3
 3 3 3 3 3 3 3 4 6 6 3 3 3 6 3 3 3 3 3 3 3 5 3 3 6 6 5 3 7 3 3 3 6 5 3 3 3
 3 3 3 3 3 3 3 3 7 3 3 3 3 6 3 3 3 3 3 3 3 8 3 3 3 3 3 6 3 3 3 5 6 3 6 3 5
 3 3 7 3 3 3 5 3 3 3 3 3 3 3 5 3 3 3 3 5 3 6 3 3 3 3 5 3 3 3 3 3 3 3 4 3 3
 3 3 3 3 3 3 6 1 5 7 3 3 3 3 3 3 3 3 3 5 3 3 3 3 6 5 3 3 3 3 3 3 3 6 5 3 3
 6 3 6 3 3 3 3 3 3 3 3 3 3 3 3 3 6 3 3 6 3 3 3 3 3 3 3 3 5 5 3 3 3 3 3 3 3
 3 7 3 3 3 3 9 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 5 3 3 3 5 6 3 7 3 3 3 3 6 3 5
 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 6 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 5 3 4 5 3 3 6 3 3 3 3 3 3 6 3 3 3 3 3 3 6 3 6 3 4 3 3 3 6
 3 3 3 6 3 5 3 3 6 3 3 6 3 7 6 3 3 5 3 3 5 6 3 3 7 3 7 3 7 3 3 3 3 3 3 3 3
 5 3 7 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3
 3 3 3 4 3 3 3 3 3 3 3 6 5 3 3 3 3 3 5 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 6 3 3 4 6 3 3 3 3 3 3 6 3 3 6 3 3 6 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 1 4 3 3 